<a href="https://colab.research.google.com/github/CurtesMalteser/text-generator-rnn/blob/master/text-generator-rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

### **Load Data**

In [3]:
with open('drive/My Drive/anna.txt', 'r') as f:
    text = f.read()

text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### **Tokenization**

Here the chars will be converted to and from integers.


In [0]:
# we create two dictionaries:
# variable names are self explanatory
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
# print encoded chars
encoded[:100]

array([ 6, 72,  4, 15, 69, 81, 53, 24, 16, 82, 82, 82, 62,  4, 15, 15, 30,
       24, 67,  4, 33, 39, 20, 39, 81, 59, 24,  4, 53, 81, 24,  4, 20, 20,
       24,  4, 20, 39, 76, 81, 63, 24, 81, 60, 81, 53, 30, 24, 46, 78, 72,
        4, 15, 15, 30, 24, 67,  4, 33, 39, 20, 30, 24, 39, 59, 24, 46, 78,
       72,  4, 15, 15, 30, 24, 39, 78, 24, 39, 69, 59, 24, 37, 22, 78, 82,
       22,  4, 30, 43, 82, 82, 34, 60, 81, 53, 30, 69, 72, 39, 78])

### **Pre-Precessing the data**
The LSTM expects as input a char converted into int, and the will be converted into one column vector where only the correspending index will have value one and remaing will be 0. This is the **one-hot encoded**.

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that one_hot_encoded works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


### **Make mini-batches**

In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### **Test Implementation**


*   Batch Size: 8
*   Sequence Steps: 50



In [0]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 6 72  4 15 69 81 53 24 16 82]
 [59 37 78 24 69 72  4 69 24  4]
 [81 78 57 24 37 53 24  4 24 67]
 [59 24 69 72 81 24 12 72 39 81]
 [24 59  4 22 24 72 81 53 24 69]
 [12 46 59 59 39 37 78 24  4 78]
 [24  2 78 78  4 24 72  4 57 24]
 [48 65 20 37 78 59 76 30 43 24]]

y
 [[72  4 15 69 81 53 24 16 82 82]
 [37 78 24 69 72  4 69 24  4 69]
 [78 57 24 37 53 24  4 24 67 37]
 [24 69 72 81 24 12 72 39 81 67]
 [59  4 22 24 72 81 53 24 69 81]
 [46 59 59 39 37 78 24  4 78 57]
 [ 2 78 78  4 24 72  4 57 24 59]
 [65 20 37 78 59 76 30 43 24 77]]


### **Define the Model**

In [11]:
# Check if the GPU is available
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print('Training on GPU!')
else:
    print('No GPU available! Training on CPU!')

Training on GPU!


In [0]:
class TextGenRNN(nn.Module):

    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

### **Train**

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: TextGenRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

### **Intantiating the Model**


In [14]:
# Define and print the net
n_hidden = 512
n_layers = 3

net = TextGenRNN(chars, n_hidden, n_layers)
print(net)

TextGenRNN(
  (lstm): LSTM(83, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [15]:
batch_size = 256
seq_length = 100
n_epochs = 40 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/40... Step: 10... Loss: 3.2107... Val Loss: 3.1169
Epoch: 1/40... Step: 20... Loss: 3.1397... Val Loss: 3.1037
Epoch: 1/40... Step: 30... Loss: 3.1461... Val Loss: 3.1005
Epoch: 1/40... Step: 40... Loss: 3.1376... Val Loss: 3.0970
Epoch: 1/40... Step: 50... Loss: 3.1276... Val Loss: 3.0960
Epoch: 1/40... Step: 60... Loss: 3.1248... Val Loss: 3.0958
Epoch: 2/40... Step: 70... Loss: 3.1344... Val Loss: 3.0945
Epoch: 2/40... Step: 80... Loss: 3.0952... Val Loss: 3.0940
Epoch: 2/40... Step: 90... Loss: 3.1065... Val Loss: 3.0940
Epoch: 2/40... Step: 100... Loss: 3.1153... Val Loss: 3.0925
Epoch: 2/40... Step: 110... Loss: 3.1000... Val Loss: 3.0838
Epoch: 2/40... Step: 120... Loss: 3.0682... Val Loss: 3.0463
Epoch: 2/40... Step: 130... Loss: 3.0203... Val Loss: 2.9890
Epoch: 3/40... Step: 140... Loss: 2.9310... Val Loss: 2.8741
Epoch: 3/40... Step: 150... Loss: 2.8680... Val Loss: 2.8234
Epoch: 3/40... Step: 160... Loss: 2.7806... Val Loss: 2.7323
Epoch: 3/40... Step: 170... Loss:

### **Checkpoint**


In [0]:
# change the name, for saving multiple files
model_name = 'rnn_40_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

### **Making Predictions**

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### **Priming and generating text**

In [0]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [36]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna, as it were, there was no one to
see about it at the poor, with his
fingers that in his stopped water was standing as she was satidfilled with him, and hoped that she had any sincerity or silent tried. All the conversation was not more than he saw
that it was not to blame. But she had telling the friend of
how all that would be badly such
a single word or take a sight of such sense of such millions as she had been said: The
princess was not all the same to the coldness of her husband, who had not succeeded, the sight of several times, he
stood as
though there was a missake that she was so thinking along in his hand. The midst was passing that he was to be superfly to him, had
studying the position of the stray the conversation and
world a peasant arouse in his sister out.

"Ah! you know what that's now
that I won't be trouble? We should not help been first to take all oud of the success of her. The carriage they were ten town in the
conversation. I won't be carting for this; I was

### **Loading a checkpoint**

In [37]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_40_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = TextGenRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [38]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Antonio said"))

And Antonio said: "Where dresses you
are," said Levin, with a table, with a cold conversation and last past.

"I always come, but I can see you to tell you, we have no strange arrival with the world of such mans of a common waltz,"
said Levin.

"Yes, but there's no day to do it," said Stepan Arkadyevitch.

"Oh, where is something to be dread?"

"Why did you talk about it, and there is such a little wife of the window when I was sincere. He decosped it, but I
were a cource of his
more and as though he would have a simply astating my subject, and that's all
have to be so much.... I've so concerned that they cannot help
making him in the same place.. I don't know how it has the same conforming house within my heart; and there was a more standing and considered all all the people, and I have stood as always too much, who came
out into such a shoulders in his hands, but she could not be as a peeple. I came over
home. Tell her that there was an instant when I can't seed take him to my mind t